In [ ]:
import diopy
import os
import pickle
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

In [ ]:
data_path = "/home/amonell/Desktop/Spatial_Plant_Pathogen_Atlas/data"

In [ ]:
seq_folder = os.path.join(data_path, "temp_objects", "AvrRpt2_alone2.h5")
scrna = diopy.input.read_h5(file=seq_folder)

In [ ]:
figure_output_folder = "6ac"

In [ ]:
# Specify the file path
file_path = os.path.join(figure_output_folder, "colors.pkl")

# Load the dictionary from the pickle file
with open(file_path, "rb") as pickle_file:
    loaded_dict = pickle.load(pickle_file)

# Print the loaded dictionary
print(loaded_dict)

In [ ]:
scrna.uns["SCT_snn_res.1_colors"] = loaded_dict

In [ ]:
scrna.uns["SCT_snn_res.1_colors"] = (
    pd.DataFrame(loaded_dict.values(), index=loaded_dict.keys())
    .loc[scrna.obs["SCT_snn_res.1"].cat.categories.values][0]
    .values
)

In [ ]:
sc.set_figure_params(dpi=400, dpi_save=400, figsize=(5, 4))
fig = sc.pl.umap(scrna, color="SCT_snn_res.1", return_fig=True)
fig.tight_layout()
fig.savefig(os.path.join(figure_output_folder, "6c_rep1_rep2.pdf"))
plt.show()
plt.close()

### 6d

In [ ]:
ad_sc = scrna.copy()
ad_sc.obs["celltype"] = (
    ad_sc.obs["celltype"]
    .replace("Epidermis", "epidermis")
    .replace("Mesophyll", "mesophyll")
    .replace("Vasculature", "vasculature")
)

In [ ]:
ad_sc = ad_sc[ad_sc.obs["celltype"] != "Unknown"]

In [ ]:
ad_sc.uns["celltype_colors"] = [
    "#FF007F",  # Bright Pink
    "#008000",  # Green
    "#FFD700",
]  # Gold

In [ ]:
sc.set_figure_params(facecolor="black", figsize=(10, 10), dpi=300)
fig = sc.pl.embedding(
    ad_sc,
    basis="umap",
    color=["celltype"],
    vmax=0.1,
    frameon=False,
    size=5,
    return_fig=True,
)

l = plt.legend()
for text in l.get_texts():
    text.set_color("black")
plt.title("celltype", {"color": "white"})
fig.savefig(os.path.join(figure_output_folder, "Figure6d_celltype_sc.pdf"))
plt.show()

In [ ]:
rep1_pseudotime = pd.read_csv(
    os.path.join(data_path, "temp_objects", "pseudotime_csv.csv")
)
scrna.obs.index = [
    i.replace("00_col_Mock_rep1", "00_Mock")
    .replace("col_AvrRpt2_24h_rep1", "AvrRpt2_24h")
    .replace("col_AvrRpt2_04h_rep1", "AvrRpt2_04h")
    .replace("col_AvrRpt2_06h_rep1", "AvrRpt2_06h")
    .replace("col_AvrRpt2_09h_rep1", "AvrRpt2_09h")
    for i in scrna.obs.index
]
new_df = scrna.obs.merge(rep1_pseudotime, how="left", left_index=True, right_on="index")
new_df.index = new_df["index"]
new_df["dpt_pseudotime"] = new_df["dpt_pseudotime"].fillna(0)
scrna.obs = new_df

In [ ]:
scrna.obs = scrna.obs[
    [
        "orig.ident",
        "nCount_RNA",
        "nFeature_RNA",
        "nCount_ATAC",
        "nFeature_ATAC",
        "nCount_SCT",
        "nFeature_SCT",
        "SCT_snn_res.1.5",
        "seurat_clusters",
        "sub_clst_rna",
        "sample.order",
        "celltype",
        "dpt_pseudotime",
    ]
]

In [ ]:
try:
    os.mkdir(os.path.join("6ac", "seq_pseudotime"))
except:
    pass

In [ ]:
sc.set_figure_params(facecolor="white", figsize=(5, 5), dpi=400, dpi_save=400)
fig = sc.pl.embedding(
    scrna,
    basis="umap",
    color="dpt_pseudotime",
    cmap="jet",
    size=50,
    show=False,
    return_fig=True,
)
fig.tight_layout()
fig.savefig(os.path.join("6ac", "seq_pseudotime", "all_batch.pdf"))

In [ ]:
scrna.obs["batch"] = [i.split("_")[-2] for i in scrna.obs["sample.order"]]
scrna.obs["rep"] = [i.split("_")[-1] for i in scrna.obs["sample.order"]]

In [ ]:
import numpy as np

sc.set_figure_params(dpi=400, dpi_save=400, figsize=(5, 5))
for sn in np.unique(scrna.obs["batch"]):
    fig = sc.pl.umap(
        scrna[(scrna.obs["batch"] == sn) & (scrna.obs.rep == "rep1")],
        color="dpt_pseudotime",
        cmap="jet",
        size=50,
        return_fig=True,
        title=sn,
    )
    fig.tight_layout()
    fig.savefig(os.path.join("6ac", "seq_pseudotime", f"{sn}.pdf"))
    plt.show()
    plt.close()